# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
#import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

#leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

In [2]:
#pd.set_option('display.max_rows', None)

df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
df[~df['Date'].str.endswith('-02-29')]
df['Date'] = pd.to_datetime(df['Date'])
start = pd.to_datetime('2005-01-01')
end = pd.to_datetime('2014-12-31')

#df_2015 = df.copy()
#df_2015 = df_2015[(df_2015['Date']> end)]
#df_2015['Data_Value'] /=10

#df = df[(df['Date'] < end)]
df['Data_Value'] /=10

df['dayofyear'] = df['Date'].dt.dayofyear

df_min = df[df['Element'] == 'TMIN']
df_max = df[df['Element'] != 'TMIN']


mintemp = df_min[df_min['Date']< end].groupby('dayofyear').aggregate({'Data_Value':np.min})
maxtemp = df_max[df_max['Date']< end].groupby('dayofyear').aggregate({'Data_Value':np.max})

mintemp.sort_values(by = 'dayofyear')
maxtemp.sort_values(by = 'dayofyear')

days = df_min.groupby('Date').min()
days = days.loc['2008'].reset_index()
days = days['Date']
days = days.dt.strftime('%b %d')
days
mintemp = pd.merge(days, mintemp, left_index=True, right_index = True)
mintemp.set_index('Date', inplace = True)
maxtemp = pd.merge(days, maxtemp, left_index=True, right_index = True)
maxtemp.set_index('Date', inplace = True)
#mintemp.index[range(0, len(mintemp), 20)]


df_2015_min = df_min[df_min['Date'] > end].groupby('dayofyear').min()
df_2015_max = df_max[df_max['Date'] > end].groupby('dayofyear').max()

df_2015_min['Date'] = df_2015_min['Date'].dt.strftime('%b %d')
df_2015_min.set_index('Date', inplace = True)
df_2015_max['Date'] = df_2015_max['Date'].dt.strftime('%b %d')
df_2015_max.set_index('Date', inplace = True)


test_m = mintemp.merge(df_2015_min, left_index = True, right_index = True)
test_m = test_m[test_m['Data_Value_x'] > test_m['Data_Value_y']]
test_m

test_max = maxtemp.merge(df_2015_max, left_index = True, right_index = True)
test_max = test_max[test_max['Data_Value_x'] < test_max['Data_Value_y']]

In [6]:
fig = plt.figure(figsize=(10, 6))
plt.plot(maxtemp['Data_Value'], label = 'Maximum temperature 2005- 2014')
plt.plot(mintemp['Data_Value'], label = 'Minimum temperature 2005- 2014')
plt.xlabel('Date')
plt.ylabel('Temperatur in C°')
plt.xticks(np.arange(12))

plt.xticks(range(0, len(mintemp), 20), mintemp.index[range(0, len(mintemp), 20)], rotation = '45')
plt.gca().fill_between(range(len(mintemp)), 
                       mintemp['Data_Value'], maxtemp['Data_Value'], 
                       facecolor='blue', alpha = 0.15)

plt.scatter(test_m.index, test_m['Data_Value_y'], color = 'blue', label = 'Min Record broken in 2015')
plt.scatter(test_max.index, test_max['Data_Value_y'], color = 'red', label = 'Max Tecord broken in 2015')
plt.legend(frameon = False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.title('Temperatures in Ann Arbor, Michigan, United States')
fig = plt.gcf()

plt.savefig('fig.svg', format ='svg')